### Initialize GPU configuration

In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.intra_op_parallelism_threads=16
config.inter_op_parallelism_threads=16
sess = tf.Session(config=config)

from keras import backend as K
K.set_session(sess)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

### Initialize some arguments

In [ ]:
# root_dir = directory of dataset images
# output_dir = where you want to save the models

image_size = 128

### Data Prep

In [ ]:
import pandas as pd
cols = ["Image Path", "Category"]
train = pd.read_csv('train-datagen.csv', header=None, names=cols)

In [ ]:
val = pd.read_csv('val-datagen.csv', header=None, names=cols)
val

## Try with batch size = 32

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 32
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# this is the augmentation configuration we will use for validation:
# only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical')  # since we use categorical_crossentropy loss

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

In [ ]:
#confirm the scaling works

batchX, batchY = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [ ]:
from keras.applications.vgg16 import VGG16
# load the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# First try by freezing all layers and adding a little batch normalization in the end

In [ ]:
# Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in base_model.layers:
    print(layer, layer.trainable)


In [ ]:
from keras import models
from keras import layers
from keras import optimizers



# Create the model
model0 = models.Sequential()
model0.add(base_model)

model0.add(layers.Flatten())
model0.add(layers.BatchNormalization())

model0.add(layers.Dense(1024, activation='relu'))
model0.add(layers.Dropout(0.3))

model0.add(layers.Dense(512, activation='relu'))
model0.add(layers.Dropout(0.5))

model0.add(layers.Dense(30, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model0.summary()


## Train the first model

In [ ]:
# Compile the model
import keras
from keras import backend as K
import tensorflow as tf
import functools 

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

model0.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc', top3_acc])

In [ ]:
# Train the model
history = model0.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1,
    max_queue_size=2000,
    workers=8)

# Save the model
model0.save(f'{output_dir}\\vggnet0.h5')  # always save your weights after training or during training

## See Performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Second try VGG by freezing all layers & adding some batch normalization & some regularizers & with batch size = 128

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 128
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# this is the augmentation configuration we will use for validation:
# only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical')  # since we use categorical_crossentropy loss

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

In [ ]:
from keras import layers
from keras import optimizers
from keras import regularizers


# Create the model
model1 = models.Sequential()
model1.add(base_model2)
model1.add(layers.Flatten())
model1.add(layers.BatchNormalization())
model1.add(layers.Dense(1024,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(layers.BatchNormalization())
model1.add(layers.Dropout(0.4))
model1.add(layers.Dense(512, activation='relu'))
model1.add(layers.BatchNormalization())
model1.add(layers.Dropout(0.4))
model1.add(layers.Dense(256,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(30, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model1.summary()

In [ ]:
# Compile the model
import keras
from keras import backend as K
import tensorflow as tf
import functools 

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

model1.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc', top3_acc])

In [ ]:
# Train the model
history = model1.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1,
    max_queue_size=2000,
    workers=8)

# Save the model
model1.save(f'{output_dir}\\vggnet1.h5')  # always save your weights after training or during training

## See Performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Third try VGG by unfreezing the first four layers & add some batch normalization & some regularizers

## Try with batch size = 128

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 128
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# this is the augmentation configuration we will use for validation:
# only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical')  # since we use categorical_crossentropy loss

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val,  # this is the target directory
        x_col = "Image Path",
        y_col = "Category",
        target_size=(image_size, image_size),  
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

In [ ]:
from keras.applications.vgg16 import VGG16

base_model2 = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [ ]:
# UnFreeze the last 4 layers
for layer in base_model2.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in base_model2.layers:
    print(layer, layer.trainable)


In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

# Create the model
model2 = models.Sequential()
model2.add(base_model2)
model2.add(layers.Flatten())
model2.add(layers.BatchNormalization())
model2.add(layers.Dense(1024,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.4))
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.4))
model2.add(layers.Dense(256,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(30, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model2.summary()

## Train the third model

In [ ]:
# Compile the model
import keras
from keras import backend as K
import tensorflow as tf
import functools 

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc', top3_acc])

In [ ]:
# Train the model
history = model2.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1,
    max_queue_size=2000,
    workers=8)

# Save the model
model2.save(f'{output_dir}\\vggnet2.h5')  # always save your weights after training or during training

## See Performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Fourth try by unfreezing the last 8 layers adding more batch normalization & kernel regularizers

In [ ]:
base_model3 = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

In [ ]:
# UnFreeze the last 4 layers
for layer in base_model3.layers[:-8]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in base_model3.layers:
    print(layer, layer.trainable)


In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

# Create the model
model3 = models.Sequential()

# Add the vgg convolutional base model
model3.add(base_model3)

# Add new layers
model3.add(layers.Flatten())
model3.add(layers.BatchNormalization())

model3.add(layers.Dense(1024,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.3))

model3.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.3))

model3.add(layers.Dense(128,  kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.4))

model3.add(layers.Dense(64, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))

model3.add(layers.Dense(30, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
model3.summary()

## Train

In [ ]:
# Compile the model
import keras
from keras import backend as K
import tensorflow as tf
import functools 

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

model3.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc', top3_acc])

In [ ]:
# Train the model
history = model3.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size ,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size,
    verbose=1,
    max_queue_size=2000,
    workers=8)

# Save the model
model3.save(f'{output_dir}\\vggnet3.h5')  # always save your weights after training or during training

## See performance

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Results:


### First Model - freezing all layers, batch size 32, some batch normalization: 
#### Validation Accuracy : 0.1637 
#### Accuracy : 0.1385

### Second Model - freezing all layers, batch size 128, more batch normalization & regularizers: 
#### Validation Accuracy : 0.231 
#### Accuracy : 0.3026 

### Third Model - unfreezing the first four layers, batch size 128, more batch normalization & regularizers: 
#### Validation Accuracy : 0.1977 
#### Accuracy : 0.8300

### Fourth Model: unfreezing the first eight layers, batch size 128, more regularizers and changing the number of hidden units:
#### Validation Accuracy : 0.1886 
#### Accuracy : 0.7069
